In [47]:
# CELL 1: Mount Google Drive and Import Libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import zipfile
from PIL import Image
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [48]:
# CELL 2: Define Paths and Constants
# ADJUST THESE PATHS
MODEL_PATH = '/content/drive/MyDrive/CornLeaf_EfficientNetv2b2.keras' # Path to saved model
TEST_ZIP_PATH = '/content/drive/MyDrive/CS180 Project Test Sets/corn_test.zip'
EXTRACT_TO_PATH = '/content/corn_test_extracted/' # Temporary path to extract test images
TEST_IMAGE_FOLDER = os.path.join(EXTRACT_TO_PATH, 'corn_test') # Path to the folder containing test images after extraction

# Model input image dimension
IMG_WIDTH = 260
IMG_HEIGHT = 260
CLASS_NAMES = ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy'] # From training

In [49]:
# CELL 3: Unzip the Test Set
if os.path.exists(TEST_IMAGE_FOLDER):
    print(f"Test images folder already exists at {TEST_IMAGE_FOLDER}. Skipping unzip.")
else:
    if os.path.exists(EXTRACT_TO_PATH):
        # Clean up previous extraction if it exists but not the specific folder
        import shutil
        shutil.rmtree(EXTRACT_TO_PATH)
    os.makedirs(EXTRACT_TO_PATH, exist_ok=True)
    with zipfile.ZipFile(TEST_ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_TO_PATH)
    print(f"Test images extracted to {TEST_IMAGE_FOLDER}")

# Verify extraction
if os.path.exists(TEST_IMAGE_FOLDER):
    print(f"Found {len(os.listdir(TEST_IMAGE_FOLDER))} files in {TEST_IMAGE_FOLDER}")
else:
    print(f"Error: {TEST_IMAGE_FOLDER} not found after attempting to extract. Please check TEST_ZIP_PATH and extraction.")

Test images folder already exists at /content/corn_test_extracted/corn_test. Skipping unzip.
Found 838 files in /content/corn_test_extracted/corn_test


In [50]:
# CELL 4: Load the Trained Model
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print("Model loaded successfully.")
    model.summary() # Verify model structure
except Exception as e:
    print(f"Error loading model: {e}")
    # Add fallback or error handling if necessary

Model loaded successfully.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 260, 260, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 260, 260, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b2 (Functional)  │ (None, 1408)           │     8,769,374 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm_top                  │ (None, 1408)           │         5,632 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ top_dropout (Dropout)           │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 4)              │         5,636 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,376,396 (58.66 MB)

 Trainable params: 3,297,876 (12.58 MB)

 Non-trainable params: 5,482,766 (20.92 MB)

 Optimizer params: 6,595,754 (25.16 MB)

In [51]:
# CELL 5: Preprocessing Function for Test Images
def preprocess_test_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((IMG_WIDTH, IMG_HEIGHT))
        img_array = tf.keras.utils.img_to_array(img)
        # EfficientNetV2B2 expects inputs in [0, 255] or uses its own preprocess_input.
        # Our training pipeline used data_augmentation directly on [0, 255] images.
        # So, for prediction, we just need to match the shape.
        # No explicit tf.keras.applications.efficientnet_v2.preprocess_input was in our training for the base model call.
        img_array = tf.expand_dims(img_array, 0) # Create a batch
        return img_array
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

In [52]:
# CELL 6: Generate Predictions
predictions_list = []
test_image_files = sorted(os.listdir(TEST_IMAGE_FOLDER), key=lambda x: int(x.split('.')[0])) # Sort by number: 0.jpeg, 1.jpeg ...

if not test_image_files:
    print(f"No image files found in {TEST_IMAGE_FOLDER}. Please check the path and extraction.")
else:
    print(f"Starting predictions for {len(test_image_files)} images...")
    for image_file in test_image_files:
        if image_file.lower().endswith(('.jpeg', '.jpg', '.png')): # Check for common image extensions
            image_path = os.path.join(TEST_IMAGE_FOLDER, image_file)
            processed_image = preprocess_test_image(image_path)

            if processed_image is not None:
                try:
                    prediction_probs = model.predict(processed_image, verbose=0)
                    predicted_class_index = np.argmax(prediction_probs[0])
                    predicted_class_name = CLASS_NAMES[predicted_class_index]
                    predictions_list.append({'id': image_file, 'label': predicted_class_name})
                except Exception as e:
                    print(f"Error predicting for image {image_file}: {e}")
                    predictions_list.append({'id': image_file, 'label': 'Error'}) # Or a default placeholder
            else:
                predictions_list.append({'id': image_file, 'label': 'PreprocessingError'})


    print(f"Finished predictions. {len(predictions_list)} predictions made.")

# Create DataFrame and save to CSV
if predictions_list:
    predictions_df = pd.DataFrame(predictions_list)
    CSV_OUTPUT_PATH = '/content/drive/MyDrive/predictions.csv' # Save to your Drive
    predictions_df.to_csv(CSV_OUTPUT_PATH, index=False)
    print(f"Predictions saved to {CSV_OUTPUT_PATH}")

    # Display first few predictions
    print("\nFirst 5 predictions:")
    print(predictions_df.head())
else:
    print("No predictions were made.")

Starting predictions for 838 images...
Finished predictions. 838 predictions made.
Predictions saved to /content/drive/MyDrive/predictions.csv

First 5 predictions:
       id    label
0  0.jpeg  Healthy
1  1.jpeg  Healthy
2  2.jpeg  Healthy
3  3.jpeg  Healthy
4  4.jpeg  Healthy
